In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import logging

# ===============================
# הגדרות
# ===============================
BASE_TAG_DIR = r"tag"
OUTPUT_EXCEL = os.path.join(BASE_TAG_DIR, "annotation_overlap.xlsx")

ANNOTATORS = {
    "Hila": {"suffix": "_hs"},
    "Lital": {"suffix": "_l"},
    "Mustafa": {"suffix": "_m"},
}

LAB_PAIRS = [
    ("lab1", "Hila", "Lital"),
    ("lab2", "Lital", "Mustafa"),
    ("lab3", "Hila", "Mustafa"),
]

IMAGE_EXTENSIONS = (".jpg", ".jpeg", ".png", ".tif", ".tiff")

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# ===============================
# פונקציות עזר
# ===============================
def normalize_name(filename, suffix):
    name = os.path.splitext(filename)[0].lower()
    if name.endswith(suffix):
        name = name[: -len(suffix)]
    return name


def load_mask(path):
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise ValueError(f"Cannot read image: {path}")
    return (mask > 128).astype(np.uint8)


def compute_metrics(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()

    area1 = mask1.sum()
    area2 = mask2.sum()

    dice = 100 * (2 * intersection / (area1 + area2)) if (area1 + area2) > 0 else 100
    iou = 100 * (intersection / union) if union > 0 else 100
    boundary_diff = 100 * ((union - intersection) / union) if union > 0 else 0

    return dice, iou, boundary_diff, intersection, union


def get_masks_map(folder, suffix):
    masks = {}
    for f in os.listdir(folder):
        if f.lower().endswith(IMAGE_EXTENSIONS):
            key = normalize_name(f, suffix)
            masks[key] = os.path.join(folder, f)
    return masks


# ===============================
# לוגיקה ראשית
# ===============================
rows = []

for lab, ann1, ann2 in LAB_PAIRS:
    logging.info(f"Processing {lab}: {ann1} vs {ann2}")

    dir1 = os.path.join(BASE_TAG_DIR, ann1, lab)
    dir2 = os.path.join(BASE_TAG_DIR, ann2, lab)

    if not os.path.isdir(dir1) or not os.path.isdir(dir2):
        logging.warning(f"Skipping {lab} – missing folder")
        continue

    masks1 = get_masks_map(dir1, ANNOTATORS[ann1]["suffix"])
    masks2 = get_masks_map(dir2, ANNOTATORS[ann2]["suffix"])

    # רק קבצים שיש להם MATCH אמיתי
    common_keys = sorted(set(masks1) & set(masks2))

    for key in common_keys:
        try:
            mask1 = load_mask(masks1[key])
            mask2 = load_mask(masks2[key])

            if mask1.shape != mask2.shape:
                logging.warning(f"Resolution mismatch: {lab}/{key}")
                continue

            dice, iou, boundary_diff, inter, union = compute_metrics(mask1, mask2)

            rows.append({
                "lab": lab,
                "image_name": key,
                "annotator_1": ann1,
                "annotator_2": ann2,
                "dice_percent": round(dice, 2),
                "iou_percent": round(iou, 2),
                "boundary_diff_percent": round(boundary_diff, 2),
                "intersection_pixels": inter,
                "union_pixels": union
            })

        except Exception as e:
            logging.warning(f"Failed processing {lab}/{key}: {e}")

# ===============================
# שמירה לאקסל
# ===============================
df = pd.DataFrame(rows)
df.to_excel(OUTPUT_EXCEL, index=False)

logging.info(f"Excel saved to: {OUTPUT_EXCEL}")
logging.info(f"Total matched rows: {len(df)}")


ModuleNotFoundError: No module named 'pandas'